In [16]:
!date
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
sns.set_context('paper')
sns.set_style('whitegrid')
pd.set_option('max_rows',10)

Thu Aug 25 21:46:17 PDT 2016


In [17]:
%cd ~/projects/2016/new_debut
import data, model

/snfs2/HOME/abie/projects/2016/new_debut


In [18]:
rep = 0
weeks_after = 12
surgery_week_start = 16
surgery_week_end = 52

In [19]:
%%time

patient_df, X, y = data.load_prepped_df(weeks_after, surgery_week_start, surgery_week_end)

CPU times: user 1min 14s, sys: 1.8 s, total: 1min 16s
Wall time: 1min 16s


In [20]:
# set random seed for reproducibility
np.random.seed(12345+rep)

In [21]:
import sklearn.naive_bayes, sklearn.ensemble, sklearn.linear_model, sklearn.model_selection

In [ ]:
# create dict of ML methods to consider
n_jobs = 10  # make sure to request corresponding resource level on cluster
clf_dict = {'GBM': sklearn.model_selection.GridSearchCV(sklearn.ensemble.GradientBoostingClassifier(),
                                                        param_grid={'max_depth':[1,3,5,7,9,11]}, n_jobs=n_jobs),
#             'NB': sklearn.model_selection.GridSearchCV(sklearn.naive_bayes.BernoulliNB(),
#                                                        param_grid={'alpha':[0, .01, .1, 1., 10., 100.]}, n_jobs=n_jobs),
#             'PLR': sklearn.linear_model.LogisticRegressionCV(n_jobs=n_jobs, class_weight='balanced'),
#             'RF': sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=n_jobs, class_weight='balanced'),
           }

In [ ]:
%%time

# run models and save results
results = {}
for clf_name, clf in clf_dict.items():
    print(clf_name, flush=True)

    cv = sklearn.model_selection.KFold(n_splits=10, shuffle=True, random_state=12345+rep)
    fold=0
    train, test = cv.split(X, y).__next__()
        
    # fit on training data
    clf.fit(X[train], y[train])

    # predict on testing data
    ypred_pr = clf.predict_proba(X[test])

    results[clf_name] = {'obs':y[test], 'pred':ypred_pr}


GBM


In [ ]:
import sklearn.metrics

In [ ]:
auc = sklearn.metrics.roc_auc_score(y[test], ypred_pr[:,-1])
print(auc)

In [ ]:
results_df = pd.DataFrame()

In [ ]:
for weeks_after in range(0,49,4):
    for surgery_week_start in range(weeks_after+4, 53, 12):
        for surgery_week_end in range(surgery_week_start+52, 104, 12):
            patient_df, X, y = data.load_prepped_df(weeks_after, surgery_week_start, surgery_week_end)
            t = model.auc_est(clf, X, y, reps=[0], folds=[0])
            t['week_after'] = weeks_after
            t['surgery_week_start'] = surgery_week_start
            t['surgery_week_end'] = surgery_week_end
            
            results_df = results_df.append(t)
            results_df.to_csv('/share/scratch/users/abie/projects/2016/TICS/auc_sweep.csv')
            print(t)

In [33]:
t

,rep,fold,clf,auc,week_after,surgery_week_start,surgery_week_end
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.49865,-4,12,88


In [34]:
results_df

,rep,fold,clf,auc,week_after,surgery_week_start,surgery_week_end
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.572605,-4,0,52
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.567529,-4,0,56
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.571065,-4,0,60
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.568141,-4,0,64
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.578847,-4,0,68
...,...,...,...,...,...,...,...
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.498655,-4,12,72
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.532897,-4,12,76
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.534878,-4,12,80
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.532911,-4,12,84


In [48]:
results_df.auc.describe()

count    103.000000
mean       0.647259
std        0.084500
min        0.498650
25%        0.573194
50%        0.644367
75%        0.729487
max        0.792798
Name: auc, dtype: float64

In [55]:
results_df.groupby(['surgery_week_start', 'surgery_week_end']).auc.last().unstack().head(10).fillna('')

surgery_week_end,52,56,60,64,68,72,76,80,84,88,92,96,100
surgery_week_start,,,,,,,,,,,,,
0,0.572605,0.567529,0.571065,0.568141,0.578847,0.584335,0.586134,0.582497,0.582941,0.579166,0.579576,0.578924,0.577101
4,,0.695454,0.573626,0.572761,0.693007,0.57183,0.567848,0.699281,0.566475,0.586146,0.685284,0.581411,0.579921
8,,,0.774432,0.547873,0.545962,0.768725,0.547626,0.549081,0.772231,0.546458,0.559588,0.765508,0.549219
12,,,,0.76662,0.500029,0.498655,0.760868,0.534878,0.532911,0.749679,,,0.748975
16,,,,,0.773806,,,0.753989,,,0.756194,,
20,,,,,,0.731537,,,0.729832,,,0.720519,
24,,,,,,,0.792798,,,0.764061,,,0.764009
28,,,,,,,,0.766535,,,0.758164,,
32,,,,,,,,,0.749595,,,0.745783,


In [51]:
results_df.sort_values('auc').tail(10)

,rep,fold,clf,auc,week_after,surgery_week_start,surgery_week_end
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.764009,20,24,100
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.764061,20,24,88
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.765508,4,8,96
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.766535,24,28,80
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.766620,8,12,64
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.768725,4,8,72
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.772231,4,8,84
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.773806,12,16,68
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.774432,4,8,60
0,0,0,"GridSearchCV(cv=None, error_score='raise',\n ...",0.792798,20,24,76
